In [8]:
pip install -U tensorflow==2.10 

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\KIIT\\anaconda3\\Lib\\site-packages\\tensorflow\\python\\_pywrap_dtensor_device.pyd'
Consider using the `--user` option or check the permissions.




  Using cached tensorflow-2.10.0-cp39-cp39-win_amd64.whl (455.9 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2


In [2]:
pip install tokenizers==0.13.3

  Using cached tokenizers-0.13.3-cp39-cp39-win_amd64.whl (3.5 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import load_metric

# Load the training and development JSON datasets
with open("C:\\Users\\KIIT\\Desktop\\CSV files\\EXIST 2024 Tweets Dataset\\training\\EXIST2024_training.json", 'r', encoding='utf-8') as file:
    train_data = json.load(file)

with open("C:\\Users\\KIIT\\Desktop\\CSV files\\EXIST 2024 Tweets Dataset\\dev\\EXIST2024_dev.json", 'r', encoding='utf-8') as file:
    dev_data = json.load(file)

# Convert the JSON data to DataFrames
def json_to_df(data):
    records = []
    for key, value in data.items():
        record = {
            'id': value['id_EXIST'],
            'lang': value['lang'],
            'tweet': value['tweet'],
            'labels_task1': value['labels_task1'],
            'split': value['split']
        }
        records.append(record)
    return pd.DataFrame(records)

train_df = json_to_df(train_data)
val_df = json_to_df(dev_data)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')

# Encode the data
def encode_data(tokenizer, df, max_length=128):
    texts = df['tweet'].tolist()
    return tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = encode_data(tokenizer, train_df)
val_encodings = encode_data(tokenizer, val_df)

# Encode labels for task1
train_labels = [1 if label == 'YES' else 0 for label in train_df['labels_task1'].apply(lambda x: x[0]).tolist()]
val_labels = [1 if label == 'YES' else 0 for label in val_df['labels_task1'].apply(lambda x: x[0]).tolist()]

# Define the Dataset class
class HinglishDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = HinglishDataset(train_encodings, train_labels)
val_dataset = HinglishDataset(val_encodings, val_labels)


C:\Users\KIIT\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Load the accuracy metric
accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/LaBSE', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results_task1',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_task1',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate on the validation set
val_results = trainer.evaluate(eval_dataset=val_dataset)
print(f"Validation Accuracy: {val_results['eval_accuracy']}")
print(f"Validation F1: {val_results['eval_f1']}")
print(f"Validation Precision: {val_results['eval_precision']}")
print(f"Validation Recall: {val_results['eval_recall']}")


C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next 

C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for precision contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/precision/precision.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for recall contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/recall/recall.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


C:\Users\KIIT\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/LaBSE and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\KIIT\anaconda3\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__cal

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.527800,0.604762,0.708092,0.708099,0.708801,0.708092
2,0.583700,0.565024,0.721580,0.720306,0.723752,0.721580
3,0.204500,0.714775,0.725434,0.723613,0.729099,0.725434


Validation Accuracy: 0.7254335260115607
Validation F1: 0.7236131688113638
Validation Precision: 0.7290988563320879
Validation Recall: 0.7254335260115607


In [6]:
from sklearn.model_selection import train_test_split

# Split the development set into validation and test sets
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)

# Encode the data
val_encodings = encode_data(tokenizer, val_df)
test_encodings = encode_data(tokenizer, test_df)

# Encode labels for task1
val_labels = [1 if label == 'YES' else 0 for label in val_df['labels_task1'].apply(lambda x: x[0]).tolist()]
test_labels = [1 if label == 'YES' else 0 for label in test_df['labels_task1'].apply(lambda x: x[0]).tolist()]

# Create dataset objects
val_dataset = HinglishDataset(val_encodings, val_labels)
test_dataset = HinglishDataset(test_encodings, test_labels)

# Evaluate on the validation set
val_results = trainer.evaluate(eval_dataset=val_dataset)
print(f"Validation Accuracy: {val_results['eval_accuracy']}")
print(f"Validation F1: {val_results['eval_f1']}")
print(f"Validation Precision: {val_results['eval_precision']}")
print(f"Validation Recall: {val_results['eval_recall']}")

# Evaluate on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test Accuracy: {test_results['eval_accuracy']}")
print(f"Test F1: {test_results['eval_f1']}")
print(f"Test Precision: {test_results['eval_precision']}")
print(f"Test Recall: {test_results['eval_recall']}")

Validation Accuracy: 0.7186897880539499
Validation F1: 0.7180142252401315
Validation Precision: 0.7235599747197653
Validation Recall: 0.7186897880539499
Test Accuracy: 0.7321772639691715
Test F1: 0.7290827408509127
Test Precision: 0.7345156104090736
Test Recall: 0.7321772639691715


In [7]:
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import load_metric
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback

# Load the training and development JSON datasets
with open("C:\\Users\\KIIT\\Desktop\\CSV files\\EXIST 2024 Tweets Dataset\\training\\EXIST2024_training.json", 'r', encoding='utf-8') as file:
    train_data = json.load(file)

with open("C:\\Users\\KIIT\\Desktop\\CSV files\\EXIST 2024 Tweets Dataset\\dev\\EXIST2024_dev.json", 'r', encoding='utf-8') as file:
    dev_data = json.load(file)

# Convert the JSON data to DataFrames
def json_to_df(data):
    records = []
    for key, value in data.items():
        record = {
            'id': value['id_EXIST'],
            'lang': value['lang'],
            'tweet': value['tweet'],
            'labels_task1': value['labels_task1'],
            'split': value['split']
        }
        records.append(record)
    return pd.DataFrame(records)

train_df = json_to_df(train_data)
val_df = json_to_df(dev_data)

# Split the development set into validation and test sets
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')

# Encode the data
def encode_data(tokenizer, df, max_length=128):
    texts = df['tweet'].tolist()
    return tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = encode_data(tokenizer, train_df)
val_encodings = encode_data(tokenizer, val_df)
test_encodings = encode_data(tokenizer, test_df)

# Encode labels for task1
train_labels = [1 if label == 'YES' else 0 for label in train_df['labels_task1'].apply(lambda x: x[0]).tolist()]
val_labels = [1 if label == 'YES' else 0 for label in val_df['labels_task1'].apply(lambda x: x[0]).tolist()]
test_labels = [1 if label == 'YES' else 0 for label in test_df['labels_task1'].apply(lambda x: x[0]).tolist()]

# Define the Dataset class
class HinglishDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = HinglishDataset(train_encodings, train_labels)
val_dataset = HinglishDataset(val_encodings, val_labels)
test_dataset = HinglishDataset(test_encodings, test_labels)

# Load the metrics
accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/LaBSE', num_labels=2)

# Updated Training Arguments
training_args = TrainingArguments(
    output_dir='./results_task1',
    num_train_epochs=5,  # Increased number of epochs
    per_device_train_batch_size=32,  # Increased batch size if hardware allows
    per_device_eval_batch_size=32,
    warmup_steps=1000,  # Increased warmup steps
    weight_decay=0.01,
    logging_dir='./logs_task1',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,  # Save only the last 2 checkpoints
    load_best_model_at_end=True,  # Load the best model at the end of training
    learning_rate=2e-5,  # Adjusted learning rate
    metric_for_best_model="accuracy",  # Monitor accuracy to save the best model
    greater_is_better=True  # Accuracy is better when higher
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer with Early Stopping Callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stopping if no improvement for 3 epochs
)

# Train the model
trainer.train()

# Evaluate on the validation set
val_results = trainer.evaluate(eval_dataset=val_dataset)
print(f"Validation Accuracy: {val_results['eval_accuracy']}")
print(f"Validation F1: {val_results['eval_f1']}")
print(f"Validation Precision: {val_results['eval_precision']}")
print(f"Validation Recall: {val_results['eval_recall']}")

# Evaluate on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test Accuracy: {test_results['eval_accuracy']}")
print(f"Test F1: {test_results['eval_f1']}")
print(f"Test Precision: {test_results['eval_precision']}")
print(f"Test Recall: {test_results['eval_recall']}")

C:\Users\KIIT\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.604900,0.628645,0.647399,0.636220,0.676338,0.647399
2,0.536000,0.612808,0.705202,0.697080,0.738193,0.705202
3,0.508700,0.568901,0.745665,0.745665,0.745665,0.745665
4,0.429100,0.632642,0.720617,0.719364,0.728153,0.720617
5,0.279600,0.702654,0.728324,0.727501,0.734160,0.728324


Validation Accuracy: 0.7456647398843931
Validation F1: 0.7456647398843931
Validation Precision: 0.7456647398843931
Validation Recall: 0.7456647398843931
Test Accuracy: 0.7360308285163777
Test F1: 0.7362963109376626
Test Precision: 0.7370042791590253
Test Recall: 0.7360308285163777


In [1]:
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import load_metric
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback

# Load the training and development JSON datasets
with open("C:\\Users\\KIIT\\Desktop\\CSV files\\EXIST 2024 Tweets Dataset\\training\\EXIST2024_training.json", 'r', encoding='utf-8') as file:
    train_data = json.load(file)

with open("C:\\Users\\KIIT\\Desktop\\CSV files\\EXIST 2024 Tweets Dataset\\dev\\EXIST2024_dev.json", 'r', encoding='utf-8') as file:
    dev_data = json.load(file)

# Convert the JSON data to DataFrames
def json_to_df(data):
    records = []
    for key, value in data.items():
        record = {
            'id': value['id_EXIST'],
            'lang': value['lang'],
            'tweet': value['tweet'],
            'labels_task1': value['labels_task1'],
            'split': value['split']
        }
        records.append(record)
    return pd.DataFrame(records)

train_df = json_to_df(train_data)
val_df = json_to_df(dev_data)

# Split the development set into validation and test sets
val_df, test_df = train_test_split(val_df, test_size=0.2, random_state=42)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')

# Encode the data
def encode_data(tokenizer, df, max_length=128):
    texts = df['tweet'].tolist()
    return tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

train_encodings = encode_data(tokenizer, train_df)
val_encodings = encode_data(tokenizer, val_df)
test_encodings = encode_data(tokenizer, test_df)

# Encode labels for task1
train_labels = [1 if label == 'YES' else 0 for label in train_df['labels_task1'].apply(lambda x: x[0]).tolist()]
val_labels = [1 if label == 'YES' else 0 for label in val_df['labels_task1'].apply(lambda x: x[0]).tolist()]
test_labels = [1 if label == 'YES' else 0 for label in test_df['labels_task1'].apply(lambda x: x[0]).tolist()]

# Define the Dataset class
class HinglishDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
train_dataset = HinglishDataset(train_encodings, train_labels)
val_dataset = HinglishDataset(val_encodings, val_labels)
test_dataset = HinglishDataset(test_encodings, test_labels)

# Load the metrics
accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained('sentence-transformers/LaBSE', num_labels=2)

# Updated Training Arguments
training_args = TrainingArguments(
    output_dir='./results_task1',
    num_train_epochs=10,  # Further increased number of epochs
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,  # Adjusted warmup steps
    weight_decay=0.005,  # Tuned weight decay parameter
    logging_dir='./logs_task1',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    learning_rate=3e-5,  # Increased learning rate slightly
    metric_for_best_model="accuracy",
    greater_is_better=True
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Trainer with Early Stopping Callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# Evaluate on the validation set
val_results = trainer.evaluate(eval_dataset=val_dataset)
print(f"Validation Accuracy: {val_results['eval_accuracy']}")
print(f"Validation F1: {val_results['eval_f1']}")
print(f"Validation Precision: {val_results['eval_precision']}")
print(f"Validation Recall: {val_results['eval_recall']}")

# Evaluate on the test set
test_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test Accuracy: {test_results['eval_accuracy']}")
print(f"Test F1: {test_results['eval_f1']}")
print(f"Test Precision: {test_results['eval_precision']}")
print(f"Test Recall: {test_results['eval_recall']}")

C:\Users\KIIT\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\KIIT\AppData\Local\Temp\ipykernel_10852\712307651.py:74: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
C:\Users\KIIT\anaconda3\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Pass

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.550300,0.602557,0.703614,0.698321,0.719980,0.703614
2,0.525200,0.573539,0.733735,0.733147,0.736165,0.733735
3,0.427300,0.597331,0.731325,0.730902,0.733091,0.731325
4,0.226800,0.836207,0.681928,0.675373,0.698959,0.681928
5,0.123300,1.160745,0.698795,0.695052,0.709867,0.698795


Validation Accuracy: 0.7337349397590361
Validation F1: 0.7331474402915423
Validation Precision: 0.7361645381789601
Validation Recall: 0.7337349397590361
Test Accuracy: 0.7115384615384616
Test F1: 0.7126152123352572
Test Precision: 0.7157913515056372
Test Recall: 0.7115384615384616
